Part 1

In [5]:
!pip install beautifulsoup4

In [6]:
!pip install lxml

In [7]:
!pip install html5lib

In [8]:
!pip install requests

In [10]:
from bs4 import BeautifulSoup
import requests

In [95]:
import numpy as np

In [82]:
import pandas as pd

column_names = ['Postcode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns = column_names)
neighborhoods

,Postcode,Borough,Neighborhood


In [89]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
page = BeautifulSoup(source, 'lxml')
toronto_table = page.table.tbody

for distinct in toronto_table.find_all('tr')[1:]:
    
    postcode = distinct.find_all('td')[0].text
    
    try:
        borough = distinct.find_all('td')[1].a.text
        
    except:
        borough = distinct.find_all('td')[1].text
    
    try:
        neighborhood = distinct.find_all('td')[2].a.text
    except:
        neighborhood = borough
    
    neighborhoods = neighborhoods.append({'Postcode': postcode,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood}, ignore_index = True)
    
neighborhoods.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [105]:
toronto_data = pd.DataFrame(neighborhoods)

In [106]:
toronto_data.drop_duplicates(['Borough','Neighborhood'], inplace = True)

In [107]:
toronto_data.replace('Not assigned', np.nan, inplace = True)
toronto_data.dropna(inplace = True)

In [134]:
toronto_data = toronto_data.groupby(['Postcode','Borough']).aggregate(lambda x: ','.join(x)).reset_index()
toronto_data.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Scarborough,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1J,Scarborough,Scarborough Village
5,M1K,Scarborough,"Ionview,Kennedy Park"
6,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
7,M1M,Scarborough,"Cliffcrest,Cliffside"
8,M1N,Scarborough,Birch Cliff
9,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He..."


In [132]:
toronto_data.shape

(84, 3)

Part 2

In [136]:
!pip install geocoder

In [137]:
import geocoder

In [143]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [144]:
coordinates.shape

(103, 3)

In [154]:
toronto_geo_data = pd.merge(toronto_data,coordinates,left_on='Postcode',right_on='Postal Code', how = 'inner', left_index = True)
toronto_geo_data.drop(labels = ['Postal Code'], axis = 1, inplace = True)
toronto_geo_data.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",43.757410,-79.273304
